# Jointure entre le fichier `mutation_geomlocmt` et `filosofi` 

Le but de ce notebook est de faire le lien entre les mutations disponibles dans la table `mutation_geomlocmut` et les informations disposnibles dans la base `Filosofi`.

Les informations géographiques dans la table `mutation_geomlocmut` sont disponibles de manière ponctuelle alors que les données issues de `filosofi`sont fournies par carreaux. Dès lors, il n'existe pas de clefs directes pour joindre ces deux tables. 

Il faut au préalable pour chaque point de la table `mutation_geomlocmut` déterminer dans quel carreau il se situe. Pour se faire, on utilise les fonctions : 

- `is_point_in_poly` : vérifie si un objet `Point` appartient à un objet `Polygon`
- `get_number_poly` : pour un objet `Point` permet de récupérer l'indice de l'objet `Polygon` auquel il appartient (None sinon).
- `add_information_to_points` : permet de faire la jointure entre les tables `mutation` et `filosofi` en utilisant l'indice du `Polygon` d'appartenance du `Point`.

Ces fonctions présentent de fortes complexités : nous avons décidé de les lancer sur Google Colab (aucune donnée confidentielle n'est mobilisée ici).



## Préparation de l'espace de travail sur Google Colab

### Lien avec Google Drive


Les données ont été importées depuis Google Drive. Elles sont disponibles en cliquant [ici](https://drive.google.com/drive/folders/1ut-i30E5ZbZRzdj42B_LNRYM4eR4_XWE?usp=sharing).

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Installation de `geopandas`

In [ ]:
pip install geopandas

In [28]:
import geopandas as gpd
import pandas as pd

### Lecture de la table `mutation`

In [26]:
dossier_donnees = "/content/drive/My Drive/projet_info_Ensae"

In [29]:
mutation_geomlocmut = gpd.read_file(dossier_donnees + "/mutation/r11_mutation_geomlocmut.shp")
mutation_geomlocmut = mutation_geomlocmut.rename(columns={"IDMUTATION" : "idmutation"})
mutation_geomlocmut_wgs = mutation_geomlocmut.to_crs({'init' :'epsg:4326'}) 

mutation_brute = pd.read_csv(dossier_donnees + "/mutation/r11_mutation.csv")

code_mutation_paris = list(mutation_brute[mutation_brute["coddep"] == 75]["idmutation"])
mutation_geomlocmut_wgs_paris = mutation_geomlocmut_wgs[mutation_geomlocmut_wgs["idmutation"].isin(code_mutation_paris)]

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


### Lecture de la table `Filosofi`

In [34]:
filosofi_paris = gpd.read_file(dossier_donnees + "filosofi/filosofi_75.shp")

In [211]:
filosofi_paris.columns

Index(['IdINSPIRE', 'Id_carr1km', 'I_est_cr', 'Id_carr_n', 'Groupe', 'Depcom',
       'I_pauv', 'Id_car2010', 'Ind', 'Men', 'Men_pauv', 'Men_1ind',
       'Men_5ind', 'Men_prop', 'Men_fmp', 'Ind_snv', 'Men_surf', 'Men_coll',
       'Men_mais', 'Log_av45', 'Log_45_70', 'Log_70_90', 'Log_ap90', 'Log_inc',
       'Log_soc', 'Ind_0_3', 'Ind_4_5', 'Ind_6_10', 'Ind_11_17', 'Ind_18_24',
       'Ind_25_39', 'Ind_40_54', 'Ind_55_64', 'Ind_65_79', 'Ind_80p',
       'Ind_inc', 'I_est_1km', 'geometry'],
      dtype='object')

## Préparation des données Filosofi

In [228]:
def recompute_data(filosofi, nom_var_zonage):
    filosofi_copie = filosofi.copy(deep = True)
    ind_par_zonage = filosofi_copie.filter(regex = "(?i)(.*(" + nom_var_zonage + "|men|log|ind|geometry).*)").groupby([nom_var_zonage]).sum() 
    #Compute proportion by age
    ind_par_zonage.loc[:,"Ind_0_3":"Ind_inc"] = ind_par_zonage.loc[:,"Ind_0_3":"Ind_inc"].apply(lambda x : 100*x /ind_par_zonage["Ind"])
    #Compute statistics on "logement"        
    nb_log = ind_par_zonage.loc[:, "Log_av45":"Log_inc"].apply(sum, axis = 1)
    ind_par_zonage.loc[:, "Log_av45":"Log_inc"] = ind_par_zonage.loc[:, "Log_av45":"Log_inc"].apply(lambda x: 100*x/nb_log) #la lambda fonction est appliquée colonne par colonne
    ind_par_zonage["Log_soc"] = 100*ind_par_zonage["Log_soc"]/nb_log
    #Compute statistics on household
    ind_par_zonage.loc[:, "Men_pauv":"Men_fmp"] = ind_par_zonage.loc[:, "Men_pauv":"Men_fmp"].apply(lambda x : 100*x / ind_par_zonage["Men"])
    ind_par_zonage.loc[:, "Men_coll":"Men_mais"] = ind_par_zonage.loc[:, "Men_coll":"Men_mais"].apply(lambda x : 100*x / ind_par_zonage["Men"])
    ind_par_zonage["nv_par_hab"] = ind_par_zonage["Ind_snv"]/ind_par_zonage["Ind"]
    ind_par_zonage["ind_par_zonage"] = ind_par_zonage["Men_surf"]/nb_log

    #Dans le cas où on n'agrège pas --> on peut ajouter l'objet Point
    if nom_var_zonage == "IdINSPIRE":
      ind_par_zonage = ind_par_zonage.merge(filosofi_copie[["IdINSPIRE", "geometry"]], on = "IdINSPIRE")
      
    return ind_par_zonage

In [231]:
filosofi_recomputed = recompute_data(filosofi_paris, "IdINSPIRE")

Association à chaque point d'un carreau


In [30]:
def is_point_in_poly(point, poly):
    return point.intersects(poly)

In [203]:
#Get the index of the poly of the point
#Return None if the point isn't in any polygon.
def get_number_poly(row_point, polys):
    point = row_point["geometry"]
    try: 
        index = list(map(lambda x : is_point_in_poly(point,x) , polys)).index(True)
        res_d = {'idmutation' : row_point["idmutation"], 'carreaux_index' : index, 'geometry' : point}
    except ValueError:
        res_d = {'idmutation' : row_point["idmutation"], 'carreaux_index' : None, 'geometry' : point}
    return res_d

In [235]:
#Tables points contient une table avec pour chaque point son idmutation (idmutation, geometry)
#polys est une table contenant les carreaux ainsi que toutes les informations liées filosfi
def add_information_to_points(tables_points, polys):
  res_temp = tables_points.apply(lambda x : get_number_poly(x, polys["geometry"]), axis = 1)
  res_temp = gpd.GeoDataFrame(res_temp.to_list())
  res = res_temp.merge(polys.drop("geometry", axis = 1), left_on = "carreaux_index", right_index = True, how = "left")
  return res

In [236]:
add_information_to_points(test_100_pts,filosofi_recomputed)

,idmutation,carreaux_index,geometry,IdINSPIRE,Ind,Men,Men_pauv,Men_1ind,Men_5ind,Men_prop,Men_fmp,Ind_snv,Men_surf,Men_coll,Men_mais,Log_av45,Log_45_70,Log_70_90,Log_ap90,Log_inc,Log_soc,Ind_0_3,Ind_4_5,Ind_6_10,Ind_11_17,Ind_18_24,Ind_25_39,Ind_40_54,Ind_55_64,Ind_65_79,Ind_80p,Ind_inc,nv_par_hab,ind_par_zonage
0,5454034,1832.0,POINT (2.34927 48.89034),CRS3035RES200mN2893200E3760800,2440.5,1421.0,17.030260,60.802252,3.377903,41.027445,6.403941,62500674.1,54967.0,99.648135,0.351865,91.062632,7.107671,0.914849,0.844476,0.070373,4.222379,4.548248,1.864372,3.298504,4.650686,4.056546,35.771358,21.798812,10.038926,9.014546,3.482893,1.475108,25609.782463,38.681914
1,5473270,917.0,POINT (2.34255 48.85683),CRS3035RES200mN2889400E3760000,130.0,82.0,13.414634,57.317073,2.439024,40.243902,4.878049,6575081.4,5579.0,96.341463,3.658537,95.121951,0.000000,0.000000,4.878049,0.000000,0.000000,1.538462,1.538462,0.000000,4.615385,3.846154,19.230769,20.000000,9.230769,30.769231,8.461538,0.769231,50577.549231,68.036585
2,5183819,571.0,POINT (2.31513 48.84359),CRS3035RES200mN2888200E3757800,1361.5,782.0,10.613811,55.882353,2.301790,44.245524,5.882353,51162449.6,38783.0,98.337596,1.662404,73.529412,8.823529,7.161125,10.358056,0.127877,6.393862,4.113111,1.395520,3.745869,4.810870,3.452075,32.317297,17.186926,12.192435,13.294161,5.361733,2.130004,37578.001910,49.594629
3,5186543,NaN,POINT (2.35657 48.89727),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5607633,978.0,POINT (2.36809 48.85908),CRS3035RES200mN2889600E3761800,1238.5,634.0,12.618297,50.315457,5.362776,43.375394,7.886435,51394429.9,41016.0,98.580442,1.419558,75.078864,1.261830,18.138801,5.362776,0.157729,5.993691,4.400484,2.018571,5.773113,6.540170,3.593056,23.334679,22.527251,12.595882,11.626968,4.521599,3.068228,41497.319257,64.694006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5738514,1022.0,POINT (2.34781 48.86034),CRS3035RES200mN2889800E3760400,1037.5,609.0,14.121511,57.142857,2.791461,32.512315,7.389163,37125360.8,28899.0,98.850575,1.149425,84.893268,0.000000,3.612479,11.330049,0.164204,10.837438,2.506024,1.060241,3.614458,4.963855,4.674699,31.036145,25.060241,11.373494,10.698795,2.506024,2.506024,35783.480289,47.453202
96,5316929,391.0,POINT (2.39342 48.84122),CRS3035RES200mN2887400E3763600,1430.5,797.0,6.649937,51.568381,2.007528,58.469260,7.026349,54160784.9,45726.0,100.000000,0.000000,19.949812,36.135508,23.337516,20.326223,0.250941,0.000000,3.739951,2.446697,3.390423,4.998252,3.984621,22.160084,16.987067,9.297448,14.959804,16.148200,1.887452,37861.436491,57.372647
97,5859855,704.0,POINT (2.37400 48.85071),CRS3035RES200mN2888600E3762200,1361.5,767.0,11.082138,53.846154,2.216428,33.898305,7.953064,46166669.2,37946.0,99.608866,0.391134,86.049544,0.130378,7.301173,6.127771,0.391134,11.473272,4.002938,1.542416,3.562248,4.333456,5.618803,33.565920,18.949688,11.825193,11.017260,3.011385,2.570694,33908.681014,49.473272
98,5334272,1584.0,POINT (2.34444 48.87881),CRS3035RES200mN2892000E3760400,2171.5,1159.0,15.012942,51.423641,2.933563,44.176014,8.541846,77430552.8,61985.0,97.584124,2.415876,81.018119,5.435720,3.623814,9.922347,0.000000,4.486626,5.019572,2.371633,5.226802,4.121575,4.743265,31.913424,20.907207,9.900990,9.532581,4.144601,2.118351,35657.634262,53.481450


In [240]:
info_mut = add_information_to_points(mutation_geomlocmut_wgs_paris, filosofi_recomputed)

In [243]:
info_mut.to_file(dossier_donnees + "/mutation/r11_mutation_geomlocmut_with_filo.shp")